In [1]:
# importing all the packages I'll need
import os
import pandas as pd
import numpy as np

# Wrangling Covid-19 Vaccination Rate from Our World in Data and Income Classification from World Bank
Raw Data:
- [OWID COVID-19 vaccine doses administered per 100 people](https://ourworldindata.org/grapher/covid-vaccination-doses-per-capita?tab=map&country=Europe~Africa~North+America~South+America~Asia~Oceania~Low+income~High+income~Lower+middle+income~Upper+middle+income)
- [World Bank Income Classification](https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups)

Objectives:
1. Load the Covid-19 vaccine rate from OWiD and World Bank income classification
2. Merge the two tables that will be used as the raw data of my analysis
3. Revamp the data to only show the monthly progress of each country's vaccination rate

### Load data

In [2]:
# Load OWiD vaccine rate data
df_vax = pd.read_csv('cvdpc.csv')

# Load World bank income classification data
df_class = pd.ExcelFile('CLASS.xlsx').parse('List of economies').dropna(how='all').copy().reset_index().drop(0)

### Checking the content of the data

In [3]:
def print_summary(title, value):
    print('-' * 80, title, '-' * 80, value, sep='\n')

def summarize_dataframe(frame):
    print_summary('Dataset Summary', frame.describe())
    print_summary('Null Values Count', frame.isnull().sum())

    for col in frame:
        print_summary(f'Unique values in column "{col}"', frame[col].unique())

In [4]:
summarize_dataframe(df_vax)

--------------------------------------------------------------------------------
Dataset Summary
--------------------------------------------------------------------------------
       total_vaccinations_per_hundred
count                    50660.000000
mean                        79.910846
std                         68.151054
min                          0.000000
25%                         14.927500
50%                         67.470000
75%                        132.770000
max                        352.440000
--------------------------------------------------------------------------------
Null Values Count
--------------------------------------------------------------------------------
Entity                               0
Code                              7439
Day                                  0
total_vaccinations_per_hundred       0
dtype: int64
--------------------------------------------------------------------------------
Unique values in column "Entity"
-----------------

In [5]:
summarize_dataframe(df_class)

--------------------------------------------------------------------------------
Dataset Summary
--------------------------------------------------------------------------------
            index
count  263.000000
mean   132.174905
std     76.316735
min      1.000000
25%     66.500000
50%    132.000000
75%    197.500000
max    264.000000
--------------------------------------------------------------------------------
Null Values Count
--------------------------------------------------------------------------------
index                    0
Economy                  0
Code                     0
Region                  46
Income group            47
Lending category       119
Other (EMU or HIPC)    205
dtype: int64
--------------------------------------------------------------------------------
Unique values in column "index"
--------------------------------------------------------------------------------
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  2

### Merging the two data

In [6]:
df_vax_class = df_vax.merge(df_class, on = "Code", how = "inner").drop(columns = ["Economy", "Lending category", "Other (EMU or HIPC)"])
df_vax_class.columns = ['Country', 'Code', 'Date', 'Vax_Rate', 'Index', 'Region', 'Income_group']

In [7]:
summarize_dataframe(df_vax_class)

--------------------------------------------------------------------------------
Dataset Summary
--------------------------------------------------------------------------------
           Vax_Rate         Index
count  41798.000000  41798.000000
mean      79.320283    103.594670
std       68.616715     60.177529
min        0.000000      1.000000
25%       14.812500     53.000000
50%       64.660000    100.000000
75%      131.907500    152.000000
max      352.440000    217.000000
--------------------------------------------------------------------------------
Null Values Count
--------------------------------------------------------------------------------
Country          0
Code             0
Date             0
Vax_Rate         0
Index            0
Region           0
Income_group    29
dtype: int64
--------------------------------------------------------------------------------
Unique values in column "Country"
---------------------------------------------------------------------------

## Group by Month

In [8]:
# obtain the country codes in the table
country_code = df_vax_class["Code"].unique()

In [9]:
# Convert the "Date" column to datetime format
df_vax_class['Date'] = pd.to_datetime(df_vax_class['Date'])

In [10]:
# create the empty data frame for the new monthly vaccination rate table
df_vax_month = pd.DataFrame()

For some reasons, if I just obtain the data for the last date of each month, the months that only contain 1 data are deleted. Therefore I re-grouped the data to obtain the last 2 dates of each month.

The code will loop each country's data and obtain the last two rows of each month.

In [11]:
for code in country_code:
    data = df_vax_class[df_vax_class["Code"].str.contains(code)]
    data = data.groupby(data["Date"].dt.month, group_keys="Code").tail(2)
    
    # Change the date time format to "yyyy=mm"
    data['Date'] = data['Date'].dt.strftime('%Y-%m') 
    df_vax_month = pd.concat([df_vax_month, data])

In [12]:
# Obtain all the year-month value in the table
vax_date = df_vax_month.Date.unique()
df_vax_month = df_vax_month.sort_values(by='Date')

In [13]:
# Save the new merged data
df_vax_month.to_csv('Cov-Vax-Income-Monthly.csv', sep = ";")

## Revamping the Data
After reinspecting the data, turns out our World in Data only show the reported covid-19 vaccination rate for each month, instead of continuously retaining the data. Therefore, I have to revamp the data so table still keeps the last data from previous month during the months the country does not report their data.

In [14]:
# Load OWiD vaccine rate data
df_vax_month = pd.read_csv('Cov-Vax-Income-Monthly.csv', sep = ";")

In [15]:
# obtain the country codes in the table
country_code = df_vax_month["Code"].unique()

# Obtain all the year-month value in the table
vax_date = df_vax_month.Date.unique()

In [16]:
# New dataframe for the revamped data
df_cont_monthly_vax = pd.DataFrame()

Now, the code will loop the data from each country and recreate a table starting from the first month they reported their latest vaccination rate until April 2022.

In [17]:
for code in country_code:
    # New dataframe to retain the latest vaccination rate data of each country
    previous_mth = pd.DataFrame()

    # Each looped country dataframe
    current_row = df_vax_month[df_vax_month["Code"].str.contains(code)]

    for mth in vax_date:
        # The year-month of each looped "Date" dataframe
        current_mth = current_row[current_row["Date"].str.contains(mth)]

        # If current month contains empty data, then the current month is filled with previous month's data
        if current_mth["Vax_Rate"].empty:
            previous_mth["Date"] = mth
            df_cont_monthly_vax = pd.concat([df_cont_monthly_vax, previous_mth])

        # Otherwise, continue adding the data
        else:
            df_cont_monthly_vax = pd.concat([df_cont_monthly_vax, current_mth])

        previous_mth = current_mth.copy()

Since when the data was grouped by the last two data from each month, so there are duplicates for each month data in the table. Therefore, the duplicate needs to be eliminated by retaining only the last data from each month, then sort it by date for further analysis and visualization.

In [18]:
df_cont_monthly_vax.drop_duplicates(subset=['Code', 'Date'], keep='last', inplace = True, ignore_index = True)
df_cont_monthly_vax = df_cont_monthly_vax.sort_values(by='Date').reset_index()

In [19]:
# Checking if the data seems ok
df_cont_monthly_vax.groupby("Date").describe()

index                                                          \
         count         mean         std   min     25%     50%      75%   
Date                                                                     
2020-12    1.0     0.000000         NaN   0.0    0.00     0.0     0.00   
2021-01    6.0    33.666667   24.929233   1.0   16.75    33.5    49.50   
2021-02    8.0    48.125000   32.817406   2.0   25.25    47.0    72.25   
2021-03   14.0   132.000000   61.905635   3.0   97.00   134.5   177.50   
2021-04   93.0   646.204301  350.147781   4.0  353.00   648.0   944.00   
2021-05  195.0  1253.297436  699.327721   5.0  655.50  1274.0  1853.00   
2021-06  199.0  1278.582915  712.796982   6.0  669.00  1296.0  1890.00   
2021-07  200.0  1285.635000  716.136804   7.0  673.00  1303.0  1900.00   
2021-08  200.0  1278.050000  725.967895   8.0  648.75  1304.0  1901.00   
2021-09  202.0  1281.569307  727.961919   9.0  656.00  1305.0  1908.00   
2021-10  202.0  1293.866337  730.166060  10.0  669.25  1318.0  1921.00   
2021-11  200.0  1308.285000  722.695529  22.0  689.75  1331.0  1928.00   
2021-12  200.0  1303.545000  728.431390  11.0  677.75  1332.0  1929.00   
2022-01  200.0  1307.680000  724.931851  12.0  691.75  1333.0  1930.00   
2022-02  200.0  1305.570000  726.119081  25.0  679.75  1334.0  1931.00   
2022-03  200.0  1312.500000  720.999634  26.0  693.75  1335.0  1932.00   
2022-04  196.0  1336.816327  709.339409  27.0  733.75  1360.0  1945.00   

                Unnamed: 0                ...  Vax_Rate          Index  \
            max      count          mean  ...       75%     max  count   
Date                                      ...                            
2020-12     0.0        1.0  25614.000000  ...    0.0000    0.00    1.0   
2021-01    68.0        6.0  23183.333333  ...    4.5975   31.20    6.0   
2021-02    95.0        8.0  22717.000000  ...    2.1550   31.20    8.0   
2021-03   222.0       14.0  21339.642857  ...   14.3775   74.53   14.0   
2021-04  1237.0       93.0  21763.494624  ...   23.1000  126.92   93.0   
2021-05  2434.0      195.0  20843.287179  ...   51.9650  229.23  195.0   
2021-06  2479.0      199.0  20806.723618  ...   74.7900  231.78  199.0   
2021-07  2490.0      200.0  20796.120000  ...   96.0175  232.88  200.0   
2021-08  2500.0      200.0  20885.400000  ...  110.7775  233.93  200.0   
2021-09  2501.0      202.0  20889.123762  ...  122.5400  236.17  202.0   
2021-10  2509.0      202.0  20804.277228  ...  131.0300  264.37  202.0   
2021-11  2510.0      200.0  20756.660000  ...  141.5300  289.69  200.0   
2021-12  2511.0      200.0  20706.885000  ...  157.6525  321.00  200.0   
2022-01  2512.0      200.0  20692.735000  ...  169.9275  329.76  200.0   
2022-02  2513.0      200.0  20683.285000  ...  183.5525  335.33  200.0   
2022-03  2514.0      200.0  20782.970000  ...  195.7950  348.27  200.0   
2022-04  2515.0      196.0  20738.663265  ...  196.1425  352.44  196.0   

                                                                     
               mean        std    min     25%    50%     75%    max  
Date                                                                 
2020-12  121.000000        NaN  121.0  121.00  121.0  121.00  121.0  
2021-01  128.500000  52.633639   55.0   95.50  128.5  172.75  187.0  
2021-02  123.000000  64.584386   21.0   79.00  128.5  185.50  192.0  
2021-03  108.214286  55.109930   21.0   68.25  109.0  136.75  202.0  
2021-04  113.053763  61.304544    1.0   61.00  121.0  168.00  210.0  
2021-05  108.076923  62.115035    1.0   53.50  108.0  162.50  217.0  
2021-06  108.507538  62.520855    1.0   53.50  108.0  163.50  217.0  
2021-07  108.390000  62.385718    1.0   53.75  107.5  163.25  217.0  
2021-08  109.035000  63.084005    1.0   53.75  107.5  165.25  217.0  
2021-09  108.990099  62.798104    1.0   54.25  107.5  164.75  217.0  
2021-10  108.341584  63.101412    1.0   53.25  107.5  164.75  217.0  
2021-11  108.135000  63.002327    1.0   53.75  106.5  164.25  217.0

#### Saving the new revamped data to be further cleaned and analyzed

In [20]:
df_cont_monthly_vax.to_csv('Cov-Vax-Income-Monthly_Continuous.csv', sep = ",", index = False)

#### Troubleshoot
Check if all dates are saved correctly

In [35]:
# Load continious monthly vaccine rate data
df_cont_monthly_vax = pd.read_csv('Cov-Vax-Income-Monthly_Continuous.csv', sep = ",")

In [36]:
# Checking for the remaining missing data to be further cleaned
missing_dates = []

for code in country_code:
    current_row = df_cont_monthly_vax[df_cont_monthly_vax["Code"].str.contains(code)]
    current_country_date_list = [new_date for new_date in current_row["Date"]]
    for date in vax_date:
        current_row = current_row[current_row["Date"].str.contains(date)]
        
        if date not in current_country_date_list:
            if date > df_vax_month[df_vax_month["Code"].str.contains(code)].values[0][3]:
                print(code, date)
                if code not in missing_dates:
                    missing_dates.append(code)
                
print(missing_dates)

[]
